<a href="https://colab.research.google.com/github/TheS1n233/Distributed-Learning-Project5/blob/experiments/other_optimezers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade torch


## Imports

In [2]:
import torch
import argparse
from torch.optim.optimizer import Optimizer, required
import torch
import torchvision
import torchvision.transforms as transforms
import random
import numpy as np
import json
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, SequentialLR, LinearLR
import matplotlib.pyplot as plt
import copy
import time
from torch.amp import GradScaler, autocast
import os
from google.colab import drive
drive.mount('/content/drive')
if not os.path.exists('/content/drive/MyDrive'):
    raise RuntimeError("Google Drive not mounted correctly!")

Mounted at /content/drive


## Model

In [3]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5)

        # Layer fully connected
        self.fc1 = nn.Linear(64 * 5 * 5, 384)
        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 100)

    def forward(self, x):
        # Layer convolutivi con ReLU e max-pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)

        # Flatten per i layer fully connected
        x = torch.flatten(x, 1)

        # Layer fully connected con ReLU
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)

        return x

## Early stopping

In [4]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=10, delta=0, path='/content/drive/MyDrive/Early2checkpoint.pt', verbose=False):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path
        self.verbose = verbose

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, optimizer=None):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict() if optimizer else None,
            'val_loss_min': val_loss
        }
        torch.save(checkpoint, self.path)
        self.val_loss_min = val_loss



## Function to get train, test and val dataset

In [5]:
def get_dataset(batch_size):
    batch_size=batch_size
    print("batch_size", batch_size)
    # Define the transform to only convert the images to tensors (without normalization yet)
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Load the CIFAR-100 training dataset
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=False, num_workers=2)

    # Initialize sums for calculating mean and std
    mean = torch.zeros(3)
    std = torch.zeros(3)

    for images, _ in train_loader:
        # Compute mean and std for each channel
        mean += images.mean(dim=[0, 2, 3])
        std += images.std(dim=[0, 2, 3])

    mean /= len(train_loader)
    std /= len(train_loader)

    print("Mean: ", mean)
    print("Std: ", std)

    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean[0].item(), mean[1].item(), mean[2].item()],
                            std=[std[0].item(), std[1].item(), std[2].item()])
    ])

    transform_test = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean[0].item(), mean[1].item(), mean[2].item()],
                            std=[std[0].item(), std[1].item(), std[2].item()])
    ])

    # Load CIFAR-100 dataset
    start_time = time.time()
    train_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=True,
        download=True,
        transform=transform_train
    )
    test_dataset = torchvision.datasets.CIFAR100(
        root='./data',
        train=False,
        download=True,
        transform=transform_test
    )
    print(f"Dataset loading time: {time.time() - start_time:.2f} seconds")

    # Split training and validation sets
    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

    # Data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    # Debugging: Check DataLoader outputs
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"Batch {i}: inputs shape: {inputs.shape}, labels shape: {labels.shape}")
        if i == 10:  # Test first 10 batches
            break
    print(f"Data loading for 10 batches completed.")
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")

    return train_loader, val_loader, test_loader

# LocalSGD

In [16]:
import torch
import copy
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, random_split


def vector_to_parameters(vector, parameters):
    pointer = 0
    for param in parameters:
        # The length of the current parameter as a flat vector
        num_param = param.numel()

        # Reshape the vector segment into the shape of the current parameter
        param.data = vector[pointer:pointer + num_param].view_as(param.data)

        # Move pointer ahead
        pointer += num_param


def local_sgd(train_dataset, val_loader, test_loader, device, num_workers, local_steps, num_epoch, batch_size, hyperparams, validation_interval, slowmo_params):

    dataset_size = len(train_dataset)
    shard_size = dataset_size // num_workers
    datasets = [
        torch.utils.data.Subset(train_dataset, range(i * shard_size, (i + 1) * shard_size))
        for i in range(num_workers)
    ]
    workers = [
        torch.utils.data.DataLoader(
            datasets[i],
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True,
        )
        for i in range(num_workers)
    ]

    total_iterations = (num_epoch * (dataset_size // batch_size)) // (num_workers * local_steps)

    # Initialize the global model and optimizer
    global_model = LeNet5().to(device)
    global_optimizer = optim.SGD(
        global_model.parameters(),
        lr=hyperparams['lr'],
        weight_decay=hyperparams['weight_decay'],
        momentum=hyperparams['momentum'],
    )


    slow_momentum = torch.zeros_like(
      torch.nn.utils.parameters_to_vector(global_model.parameters())
    )

    scheduler = CosineAnnealingLR(global_optimizer, T_max=num_epoch)
    criterion = nn.CrossEntropyLoss()

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    for iteration in range(1, total_iterations + 1):

        initial_params = torch.nn.utils.parameters_to_vector(global_model.parameters()).clone()

        print(f"Iteration {iteration}/{total_iterations}")

        # Local training
        local_models = []
        worker_iters = [iter(workers[i]) for i in range(num_workers)]
        worker_train_losses = []
        worker_train_accuracies = []

        for worker_id in range(num_workers):
            local_model = copy.deepcopy(global_model).to(device)
            local_optimizer = optim.SGD(
                local_model.parameters(),
                lr=hyperparams['lr'],
                weight_decay=hyperparams['weight_decay'],
                momentum=hyperparams['momentum'],
            )

            worker_loss, correct, total = 0.0, 0, 0
            for _ in range(local_steps):
                try:
                    # Get the next batch from the worker's dataset
                    inputs, labels = next(worker_iters[worker_id])
                except StopIteration:
                    # If all data has been traversed, reinitialize the iterator
                    worker_iters[worker_id] = iter(workers[worker_id])
                    inputs, labels = next(worker_iters[worker_id])

                inputs, labels = inputs.to(device), labels.to(device)
                local_optimizer.zero_grad()
                outputs = local_model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                local_optimizer.step()

                worker_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)

            worker_loss /= local_steps
            worker_accuracy = correct / total
            worker_train_losses.append(worker_loss)
            worker_train_accuracies.append(worker_accuracy)

            local_models.append(local_model)


        averaged_params = torch.zeros_like(torch.nn.utils.parameters_to_vector(global_model.parameters()))
        for local_model in local_models:
            averaged_params += torch.nn.utils.parameters_to_vector(local_model.parameters())
        averaged_params /= num_workers


        gamma = hyperparams['lr']
        slow_momentum = slowmo_params['beta'] * slow_momentum + (1/gamma) * (initial_params - averaged_params)

        # Update outer iterates: x_{t+1,0} = x_t,0 - α*γ*u_{t+1}
        new_params = initial_params - slowmo_params['alpha'] * gamma * slow_momentum

        # Load updated parameters back into global model
        vector_to_parameters(new_params, global_model.parameters())


        global_train_loss = sum(worker_train_losses) / num_workers
        global_train_accuracy = sum(worker_train_accuracies) / num_workers


          # Validate once after "validation_interval"
        if iteration % validation_interval == 0 or iteration == total_iterations:
            # Record training metrics
            train_losses.append(global_train_loss)
            train_accuracies.append(global_train_accuracy)

            # Record val metrics
            global_model.eval()
            val_loss, correct, total = 0.0, 0, 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = global_model(inputs)
                    loss = criterion(outputs, labels)
                    val_loss += loss.item()
                    _, predicted = outputs.max(1)
                    correct += predicted.eq(labels).sum().item()
                    total += labels.size(0)

            val_loss /= len(val_loader)
            val_acc = correct / total
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)
            print(f"[Train @ Iter {iteration}] Loss: {global_train_loss:.4f}, Accuracy: {global_train_accuracy * 100:.2f}%")
            print(f"[Validation @ Iter {iteration}] Loss: {val_loss:.4f}, Accuracy: {val_acc * 100:.2f}%")

        scheduler.step()

    # Test evaluation
    global_model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = global_model(inputs)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total
    print(f"Test Accuracy: {test_acc * 100:.2f}")

    return train_losses, val_losses, train_accuracies, val_accuracies, test_acc, total_iterations


# Visualization function
def plot_results(train_losses, val_losses, train_accuracies, val_accuracies, validation_interval, total_iterations):
    # Dynamic calculation record points
    recorded_iterations = len(train_losses)
    iterations = range(validation_interval, validation_interval * recorded_iterations + 1, validation_interval)

    # Ensure data length is consistent
    assert len(train_losses) == len(iterations), f"Train losses and iterations mismatch: {len(train_losses)} vs {len(iterations)}"
    assert len(val_losses) == len(iterations), f"Validation losses and iterations mismatch: {len(val_losses)} vs {len(iterations)}"

    plt.figure(figsize=(12, 5))

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(iterations, train_losses, label="Train Loss")
    plt.plot(iterations, val_losses, label="Validation Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(iterations, train_accuracies, label="Train Accuracy")
    plt.plot(iterations, val_accuracies, label="Validation Accuracy")
    plt.xlabel("Iterations")
    plt.ylabel("Accuracy (%)")
    plt.title("Training and Validation Accuracy")
    plt.legend()

    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    num_epoch = 150
    batch_size = 64
    num_workers_list = [2, 4, 8]
    local_steps_list = [4, 8, 16, 32, 64]
    validation_interval = 15
    hyperparams = {
        'lr': 0.01,
        'weight_decay': 0.001,
        'momentum': 0.9,
    }


    slowmo_params = {
        'alpha': 1.0,
        'beta': 0.6
    }


    train_loader, val_loader, test_loader = get_dataset(batch_size)
    train_dataset = train_loader.dataset

    for num_workers in num_workers_list:
        for local_steps in local_steps_list:
            print(f"Running LocalSGD with {num_workers} workers and {local_steps} local steps")
            train_losses, val_losses, train_accuracies, val_accuracies, test_acc, total_iterations = local_sgd(
                train_dataset=train_dataset,
                val_loader=val_loader,
                test_loader=test_loader,
                device=device,
                num_workers=num_workers,
                local_steps=local_steps,
                num_epoch=num_epoch,
                batch_size=batch_size,
                hyperparams=hyperparams,
                validation_interval=validation_interval,
                slowmo_params=slowmo_params
            )

            plot_results(train_losses, val_losses, train_accuracies, val_accuracies, validation_interval, total_iterations)


batch_size 64
Files already downloaded and verified
Mean:  tensor([0.5070, 0.4865, 0.4408])
Std:  tensor([0.2664, 0.2555, 0.2750])
Files already downloaded and verified
Files already downloaded and verified
Dataset loading time: 1.74 seconds
Batch 0: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 1: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 2: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 3: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 4: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 5: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 6: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 7: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 8: inputs shape: torch.Size([64, 3, 32, 32]), labels shape: torch.Size([64])
Batch 9: in

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Iteration 2/11718
Iteration 3/11718
Iteration 4/11718
Iteration 5/11718
Iteration 6/11718
Iteration 7/11718
Iteration 8/11718
Iteration 9/11718
Iteration 10/11718
Iteration 11/11718
Iteration 12/11718
Iteration 13/11718
Iteration 14/11718
Iteration 15/11718
[Train @ Iter 15] Loss: 4.5960, Accuracy: 1.56%
[Validation @ Iter 15] Loss: 4.5996, Accuracy: 1.31%
Iteration 16/11718
Iteration 17/11718
Iteration 18/11718
Iteration 19/11718
Iteration 20/11718
Iteration 21/11718
Iteration 22/11718
Iteration 23/11718
Iteration 24/11718
Iteration 25/11718
Iteration 26/11718
Iteration 27/11718
Iteration 28/11718
Iteration 29/11718
Iteration 30/11718
[Train @ Iter 30] Loss: 4.5770, Accuracy: 2.15%
[Validation @ Iter 30] Loss: 4.5809, Accuracy: 1.98%
Iteration 31/11718
Iteration 32/11718
Iteration 33/11718
Iteration 34/11718
Iteration 35/11718
Iteration 36/11718
Iteration 37/11718
Iteration 38/11718
Iteration 39/11718
Iteration 40/11718
Iteration 41/11718
Iteration 42/11718
Iteration 43/11718
Iteratio

KeyboardInterrupt: 